In [ ]:
import requests
import json
# from csv import reader 
import os
import sys
from bs4 import BeautifulSoup
import time
import random
from lxml import etree
import os
import re
import datetime
# # from selenium import webdriver


In [ ]:
nowTime = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

In [ ]:
# year = ['2014','2015','2016','2017','2018','2019','2020','2021','2022']
domain = 'https://www.caranddriver.com'
# headers = {
#         'Accept': '*/*',
#         'Accept-Encoding': 'gzip, deflate, br',
#         'Host':'www.kbb.com',
#         'Referer':'https://www.kbb.com/car-reviews/',
#         'Content-Type':'application/json',
#         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
#         }

In [466]:
make=[
# 'Acura'#,
# 'Alfa-Romeo'#,
# 'Aston-Martin'#,
# 'Audi'#,
# 'Bentley'#,
# 'BMW'#,
# 'Bugatti'#,
# 'Buick'#,
# 'Cadillac'#,
# 'Chevrolet'#,
# 'Chrysler'#,
# 'Dodge'#,
# 'Ferrari'#,
# 'Fiat'#,
# 'Ford'#,
# 'Genesis'#,
# 'GMC'#,
# 'Honda'#,
# 'Hyundai'#,
# 'Infiniti'#,
# 'Jaguar'#, # 无年份
# 'Jeep'#,
# 'Kia'#,
# 'Lamborghini'#,
# 'Land-Rover'#,
# 'Lexus'#,
# 'Lincoln'#,
# 'Lotus'#,
# 'Maserati'#,
# 'Mazda'#,
# 'McLaren'#,
# # 'Mercedes-AMG',
# 'Mercedes-Benz'#,
# # 'Mercedes-Maybach',
# 'Mini'#,
# 'Mitsubishi'#,
# 'Nissan'#,
# 'Polestar',
# 'Porsche',# 有问题
'Ram', # 无年份
'Rolls-Royce',
'Scion',
'Smart',
'Subaru',
'Suzuki',
'Tesla',
'Toyota',
'Volkswagen',
'Volvo'
]

In [ ]:
def get_specs(url):
    response = requests.get(url=url)
    soup= BeautifulSoup(response.text, "lxml")
    specs={}
    for i in soup.select('div[class="css-qgjj1l e17ofjz23"]'):
        # print(i.select('button[class="css-17t01md e17ofjz22"]')[0].text)
        specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text]={}
        list=[]
        for j in i.select('div[class="css-1ajawdl e2zahha0"]'):
            try:
                specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text][j.contents[0].text]=j.contents[1].text
                # print(j.contents[0].text,j.contents[1].text)
            except IndexError:
                list.append(j.contents[0].text)
                # print(j.contents[0].text)
                specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text]=list
    return specs
def get_rates(url):
    response = requests.get(url=url)
    soup= BeautifulSoup(response.text, "lxml")
    # html = etree.HTML(response.text)
    # html.xpath()
    rate={}
    try:
        star={}
        for i in soup.select('div[class="eov6l8h1 css-4dxonz ehh6yb86"]'):
            star[i.select('span[class="css-ihxznl ehh6yb82"]')[0].text]=i.select('span[class="css-1c6thby ehh6yb83"]')[0].text
        mark={}
        for i in soup.select('div[class="css-12jjwzq"]'):
            mark[i.select('div[class="css-1rttn8x"]')[0].text]=i.select('div[class="css-sf59yt"]')[0].text
            # print(i.select('div[class="css-1rttn8x"]')[0].text,i.select('div[class="css-sf59yt"]')[0].text)
        rate={
        'overall':soup.select('div[class="css-wti69m"]')[0].text,
        'count':re.findall('.*\([^\)\(\d]*(\d+)[^\)\(\d]*\).*',soup.select('div[class="css-14z3b1p"]')[0].text)[0],
        'recby': soup.select('span[class="css-19pqkoc e1agtnah1"]')[0].text,
        'star':star,
        'mark':mark
        }
    except IndexError:
        pass
    return rate

In [467]:
# get url for Specs & Rate
for m in make:
    url_main=domain+'/'+m.lower()
    response = requests.get(url=url_main)
    soup=soup= BeautifulSoup(response.text, "lxml")
    href=soup.select('a[class="vehicle-item-title item-title"]')
    vehicle={}
    for u in href:
        url_each=domain+u.get('href')
        print(url_each)
        response = requests.get(url=url_each)
        soup= BeautifulSoup(response.text, "lxml")
        name=soup.select('h1[class="css-i4j13t e10ise8i2"]')[0].text
        try:
            year=soup.select('span[itemprop="name"]')[3].text
        except IndexError:
            year=soup.select('a[class^="current"]')[0].text
        make=soup.select('span[itemprop="name"]')[1].text
        model=soup.select('span[itemprop="name"]')[2].text
        # price=soup.select('div[class="css-48aaf9 e1zcv6h1"]')[0].text
        kbb_url='https://www.kbb.com'+'/'+make.replace(' ','-').lower()+'/'+model.replace(' ','-').lower()+'/'+year+'/consumer-reviews'
        vehicle[name]={
            'year':year,
            'make':make,
            'model':model}
        try:
            price=re.findall('\d+',soup.select('span[class="css-1ykuyyb e10ise8i0"]')[0].text.replace(',',''))[0]
        except IndexError:
            price=''
        vehicle[name]['price']=price
        vehicle[name]['kbb_url']=kbb_url
        vehicle[name]['rate']=get_rates(kbb_url)
        try:
            cnd_url=domain+soup.select('#main-content > header > div.css-16uv341.endngjl0 > div.css-70qvj9.e17a0kax2 > div > a[title="Specs"]')[0].attrs['href']
            vehicle[name]['cnd_url']=cnd_url
            vehicle[name]['specs']=get_specs(cnd_url)
        except IndexError:
            vehicle[name]['cnd_url']=''
            vehicle[name]['specs']={}

    with open(m+'.json', "w") as dump_f:
        json.dump(vehicle, dump_f, indent=4)

https://www.caranddriver.com/ram/1500-ev
https://www.caranddriver.com/ram/promaster
https://www.caranddriver.com/ram/1500
https://www.caranddriver.com/ram/1500-trx
https://www.caranddriver.com/ram/2500-3500
https://www.caranddriver.com/ram/promaster-2022
https://www.caranddriver.com/ram/promaster-city
https://www.caranddriver.com/ram/1500-2021
https://www.caranddriver.com/ram/1500-trx-2021
https://www.caranddriver.com/ram/2500-3500-2021
https://www.caranddriver.com/ram/promaster-2021
https://www.caranddriver.com/ram/promaster-city-2021
https://www.caranddriver.com/ram/1500-2020
https://www.caranddriver.com/ram/2500-3500-2020
https://www.caranddriver.com/ram/promaster-2020
https://www.caranddriver.com/ram/promaster-city-2020
https://www.caranddriver.com/ram/1500-2019
https://www.caranddriver.com/ram/2500-3500-2019
https://www.caranddriver.com/ram/promaster-2019
https://www.caranddriver.com/ram/promaster-city-2019
https://www.caranddriver.com/ram/1500-2018
https://www.caranddriver.com/ra

In [ ]:
with open('Ford.json', "w") as dump_f:
    json.dump(vehicle, dump_f, indent=4)

In [ ]:
# for test
url='https://www.caranddriver.com/bmw'
response = requests.get(url=url)
soup= BeautifulSoup(response.text, "lxml")

In [ ]:
#latest vehicle
renew_dict={}
renew_dict={
'name':soup.select('div[class="vehicle-item tower-ad-story"] a[class="vehicle-item-title item-title"]')[0].text,
'latest_url':domain+soup.select('div[class="vehicle-item tower-ad-story"] a[class="vehicle-item-title item-title"]')[0].get('href')
}

In [ ]:
#get each model from car&driver
soup.select('select[id="styleSelect"] option[value!="0"]:not([selected=""])')
str='https://www.kbb.com/fiat/500x/2022/consumer-reviews'
re.findall('^(.*[\\\/])',str) #获取/后所有
# re.findall('[^/]+(?!.*/)',str)#获取/前所有
year=re.findall('(^\d{4})',name)[0]
make=re.findall('[^ ]+',name)[1]
model=' '.join(re.findall('[^ ]+',name)[2:])

In [ ]:
str='(2022222 reviews)'
re.findall('.*\([^\)\(\d]*(\d+)[^\)\(\d]*\).*',str)
re.findall('.*\([^\)\(\d]*(\d+)[^\)\(\d]*\).*',str)

In [ ]:
jsonPath=os.path
with open(jsonPath,'r',encoding = 'utf-8') as load_f:
       load_dict = json.load(load_f)
       #将获取到的appToken覆盖到data_config下datajson.json文件对应的操作中,不是登陆
       load_dict[jsonkey]['appToken'] = res['data']['appToken']
with open(jsonPath,'w',encoding = 'utf-8') as dump_f:
       json.dump(load_dict,dump_f,indent = 3,sort_keys = True)

In [ ]:
str='$2,990,000'.replace(',','')
re.findall('\d+',str)